In [7]:
import psycopg2
import requests

# Connect to local db
conn = psycopg2.connect(
    host='localhost',
    port=5432,
    dbname='postgres',
    user='postgres',
    password='postgres'
)

# Create cursor
cur = conn.cursor()

# Create table for currency rates if not exists
cur.execute("CREATE TABLE IF NOT EXISTS currency_rates (id SERIAL PRIMARY KEY, date DATE, currency VARCHAR(3), rate FLOAT)")

# List of currencies to get rates for
currencies = ['AZN', 'EUR', 'KZT', 'USD', 'UZS']

# Get today's date
import datetime
today = datetime.datetime.today().strftime('%d-%m-%Y')

# Convert date to correct format
converted_date = datetime.datetime.strptime(today, '%d-%m-%Y').strftime('%Y-%m-%d')

# API endpoint
url = f"http://www.cbr.ru/scripts/XML_daily.asp?date_req={today}"

# Get data from API
data = requests.get(url).text

# Parse data
import xml.etree.ElementTree as ET
root = ET.fromstring(data)

# Insert data into db
for child in root.findall(".//Valute"):
    if child.find("CharCode").text in currencies:
        cur.execute("INSERT INTO currency_rates (date, currency, rate) VALUES (%s, %s, %s)", (converted_date, child.find("CharCode").text, float(child.find("Value").text.replace(",", "."))/float(child.find("Nominal").text)))

# Commit changes
conn.commit()

# Close cursor and connection
cur.close()
conn.close()
